In [1]:
!pip install lightgbm
!pip install optuna
!pip install snowflake-connector-python
!pip install python-dotenv

  Using cached optuna-4.3.0-py3-none-any.whl.metadata (17 kB)
  Using cached colorlog-6.9.0-py3-none-any.whl.metadata (10 kB)
Using cached optuna-4.3.0-py3-none-any.whl (386 kB)
Using cached colorlog-6.9.0-py3-none-any.whl (11 kB)
  Using cached asn1crypto-1.5.1-py2.py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 988.6/988.6 kB 14.5 MB/s eta 0:00:00
Using cached asn1crypto-1.5.1-py2.py3-none-any.whl (105 kB)


In [2]:
# --- 0. Imports and Environment Setup ---
import pandas as pd
import numpy as np
import lightgbm as lgb
import optuna
import pickle
from datetime import timedelta
from dotenv import load_dotenv
import os
import snowflake.connector

# Load environment variables from .env file
load_dotenv()

# --- 1. Snowflake Connection Setup ---
sf_account = os.getenv("SNOWFLAKE_ACCOUNT")
sf_user = os.getenv("SNOWFLAKE_USER")
sf_password = os.getenv("SNOWFLAKE_PASSWORD")
sf_warehouse = os.getenv("SNOWFLAKE_WAREHOUSE")
sf_database = os.getenv("SNOWFLAKE_DATABASE")
sf_schema = os.getenv("SNOWFLAKE_SCHEMA")
sf_role = os.getenv("SNOWFLAKE_ROLE")

def get_snowflake_conn():
    return snowflake.connector.connect(
        user=sf_user,
        password=sf_password,
        account=sf_account,
        warehouse=sf_warehouse,
        database=sf_database,
        schema=sf_schema,
        role=sf_role,
    )

# Pickup Location ID	Zone Name	Borough
# 132	JFK Airport	Queens
# 237	Upper East Side South	Manhattan
# 161	Lincoln Square East	Manhattan
# 43	Grand Central	Manhattan

pickup_location_id = 43

# --- 2. Feature Engineering Function ---
def make_features(df):
    df = df.copy()
    df['RIDES'] = pd.to_numeric(df['RIDES'], errors='coerce')
    df['hour'] = df.index.hour
    df['dayofweek'] = df.index.dayofweek

    rides = df['RIDES'].values
    n_hours_in_week = 7 * 24
    week_offsets = [n_hours_in_week * i for i in range(1, 5)]
    hour_offsets = list(range(-5, 6))

    lag_feature_names = []
    lag_features = []
    for week_num, week_offset in enumerate(week_offsets, 1):
        for h_offset in hour_offsets:
            lag = week_offset + h_offset
            lag_feature_names.append(f'lag_w{week_num}_h{h_offset:+d}')
            lag_features.append(np.roll(rides, lag))

    lag_features = np.column_stack(lag_features)
    lag_df = pd.DataFrame(lag_features, index=df.index, columns=lag_feature_names)

    max_lag = max(week_offsets) + max(abs(h) for h in hour_offsets)
    lag_df.iloc[:max_lag, :] = np.nan

    mean_4w_same_hour = lag_df[[f'lag_w{i}_h+0' for i in range(1, 5)]].mean(axis=1)
    df['mean_4w_same_hour'] = mean_4w_same_hour

    df = pd.concat([df, lag_df], axis=1)
    return df, lag_feature_names

# --- 3. Fetch Training Data ---
query = f"""
SELECT pickup_hour, rides
FROM NYC_DATA.PUBLIC.YELLOW_TAXI_DATA_TRANSFORMED
WHERE PICKUP_LOCATION_ID = {pickup_location_id}
  AND pickup_hour >= '2023-01-01'
  AND pickup_hour < '2024-02-01'
ORDER BY pickup_hour
"""

conn = get_snowflake_conn()
df = pd.read_sql(query, conn)
conn.close()
df['PICKUP_HOUR'] = pd.to_datetime(df['PICKUP_HOUR'])
df = df.set_index('PICKUP_HOUR').sort_index()

# --- 4. Feature Engineering for Training ---
df, lag_feature_names = make_features(df)
df['target'] = df['RIDES'].shift(-1)
df = df.dropna()

features = ['RIDES', 'hour', 'dayofweek', 'mean_4w_same_hour'] + lag_feature_names
X = df[features].astype(np.float32)
y = df['target'].astype(np.float32)

# --- 5. Train/Test Split (last 4 weeks as test) ---
split_point = df.index.max() - timedelta(weeks=4)
X_train, X_test = X[X.index <= split_point], X[X.index > split_point]
y_train, y_test = y[y.index <= split_point], y[y.index > split_point]

# --- 6. Optuna Objective Function ---
def objective(trial):
    params = {
      "objective": "regression",
      "metric": "mae",
      "num_leaves": trial.suggest_int("num_leaves", 30, 90),
      "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.03, log=True),
      "n_estimators": trial.suggest_int("n_estimators", 350, 900),
      "min_child_samples": trial.suggest_int("min_child_samples", 80, 130),
      "subsample": trial.suggest_float("subsample", 0.9, 1.0),
      "colsample_bytree": trial.suggest_float("colsample_bytree", 0.7, 0.9),
      "reg_alpha": trial.suggest_float("reg_alpha", 0.8, 2.0),
      "reg_lambda": trial.suggest_float("reg_lambda", 1.0, 2.0),
      "random_state": 42,
      "verbosity": -1,
      "n_jobs": -1
    }
    model = lgb.LGBMRegressor(**params)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    mae = np.mean(np.abs(preds - y_test))
    return mae

# --- 7. Run Optuna and Save Model ---
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=200, n_jobs=1)
best_params = study.best_params

# Train final model on all data
final_model = lgb.LGBMRegressor(**best_params, n_jobs=-1)
final_model.fit(X, y)

# Predict and evaluate
preds = final_model.predict(X_test)
preds_ceil = np.ceil(preds).astype(int)
mae = np.mean(np.abs(preds_ceil - y_test))
print("MAE:", mae)

# Save model as pickle
model_path = "lgbm_nyc_taxi.pkl"
with open(model_path, "wb") as f:
    pickle.dump(final_model, f)

print("Best params:", best_params)
print(f"Model saved to {model_path}")

/var/folders/hw/6940phmn1lz8gtmdwt62nz980000gn/T/ipykernel_54661/1417125309.py:10: DeprecationWarning: This package has been renamed to snowflake_uuid and will be removed shortly. Please update immediately.
  import snowflake.connector


ModuleNotFoundError: No module named 'snowflake.connector'; 'snowflake' is not a package

In [ ]:

# --- Fetch Data for Batch Prediction ---
query = f"""
SELECT pickup_hour, rides
FROM NYC_DATA.PUBLIC.YELLOW_TAXI_DATA_TRANSFORMED
WHERE PICKUP_LOCATION_ID = {pickup_location_id}
  AND pickup_hour >= '2023-12-01'
ORDER BY pickup_hour
"""

conn = get_snowflake_conn()
df = pd.read_sql(query, conn)
df['PICKUP_HOUR'] = pd.to_datetime(df['PICKUP_HOUR'])
df = df.set_index('PICKUP_HOUR').sort_index()

# --- Feature Engineering ---
df, lag_feature_names = make_features(df)

# --- Prepare Features for Prediction ---
features = ['RIDES', 'hour', 'dayofweek', 'mean_4w_same_hour'] + lag_feature_names
df_features = df[features].astype(np.float32)
df_features = df_features.dropna()  # Only predict where all features are available

# --- Load Trained Model ---
model_path = "lgbm_nyc_taxi.pkl"
with open(model_path, "rb") as f:
    model = pickle.load(f)

# --- Batch Predict ---
preds = model.predict(df_features)
preds_ceil = np.ceil(preds).astype(int)

# --- Prepare Results DataFrame ---
prediction_times = df_features.index + pd.Timedelta(hours=1)
result_df = pd.DataFrame({
    "pickup_hour": prediction_times.astype(str),
    "year": prediction_times.year,
    "month": prediction_times.month,
    "day": prediction_times.day,
    "hour": prediction_times.hour,
    "predicted_rides": preds_ceil,
    "pickup_location_id": pickup_location_id
})

# --- Write Predictions to Snowflake ---
table_name = "NYC_DATA.PUBLIC.YELLOW_TAXI_DATA_PREDICTIONS"

with get_snowflake_conn() as conn:
    with conn.cursor() as cur:
        # Create the table if it doesn't exist
        cur.execute(f"""
            CREATE TABLE IF NOT EXISTS {table_name} (
                pickup_hour TIMESTAMP,
                year INTEGER,
                month INTEGER,
                day INTEGER,
                hour INTEGER,
                predicted_rides INTEGER,
                pickup_location_id INTEGER
            )
        """)
        # Delete existing predictions for this pickup_location_id
        delete_sql = f"""
            DELETE FROM {table_name}
            WHERE pickup_location_id = %s
        """
        cur.execute(delete_sql, (pickup_location_id,))

        # Insert all predictions
        insert_sql = f"""
            INSERT INTO {table_name} (pickup_hour, year, month, day, hour, predicted_rides, pickup_location_id)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """
        cur.executemany(insert_sql, result_df.values.tolist())

print("Batch predictions written to Snowflake table:", table_name)

<ipython-input-15-74ba53b672ca>:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Batch predictions written to Snowflake table: NYC_DATA.PUBLIC.YELLOW_TAXI_DATA_PREDICTIONS
